<a href="https://colab.research.google.com/github/Pugianf/Big_Data_and_Public_Sector_II/blob/main/Trabalho_Final_Big_Data_e_Setor_P%C3%BAblico_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Questão 1

### 1.1 - Leitura dos Dados

In [40]:
# intalando as bibliotecas necessárias

!pip install statsmodels
!pip install linearmodels
!pip unintall scipy
!pip install --upgrade scipy

from IPython.display import clear_output 

clear_output()

In [71]:
# importando as bibliotecas necessárias

import pandas as pd
import numpy as np
import statistics
from scipy import stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import clear_output 

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
# módulos de teste

from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import het_breuschpagan, linear_reset
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson
from patsy import dmatrices

In [43]:
# lendo os dados e criando o DataFrame

df = pd.read_stata('/content/drive/MyDrive/IDP/Dataset/evaluation.dta')

In [45]:
# visualizando o shape da base de dados

df.shape

(19827, 22)

In [46]:
# primeira vista da base de dados

df.head(10)

,locality_identifier,household_identifier,treatment_locality,promotion_locality,eligible,enrolled,enrolled_rp,poverty_index,round,health_expenditures,age_hh,age_sp,educ_hh,educ_sp,female_hh,indigenous,hhsize,dirtfloor,bathroom,land,hospital_distance,hospital
0,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,0.0,15.185455,24.0,23.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
1,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,1.0,19.580902,25.0,24.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
2,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,0.0,13.076257,30.0,26.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,0.0
3,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,1.0,2.398854,31.0,27.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,1.0
4,26.0,13.0,1.0,1.0,1.0,1.0,0.0,54.095825,1.0,0.000000,59.0,57.0,0.0,0.0,0.0,0.0,6.0,1,0,4,124.819966,1.0
5,26.0,13.0,1.0,1.0,1.0,1.0,0.0,54.095825,0.0,15.286353,58.0,56.0,0.0,0.0,0.0,0.0,6.0,1,0,4,124.819966,0.0
6,26.0,16.0,1.0,1.0,1.0,1.0,1.0,56.903400,1.0,20.026909,36.0,25.0,3.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
7,26.0,16.0,1.0,1.0,1.0,1.0,1.0,56.903400,0.0,11.311761,35.0,24.0,3.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
8,26.0,21.0,1.0,1.0,1.0,1.0,1.0,46.908810,0.0,11.223912,37.0,35.0,0.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
9,26.0,21.0,1.0,1.0,1.0,1.0,1.0,46.908810,1.0,16.664686,39.0,36.0,0.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0


In [47]:
# vendo a descrição das variáveis (só funciona com arquivos .dta)

# lendo novamente a base, mas agora com iterator, o que retorna um objeto do tipo StataReader

stata_reader = pd.read_stata('/content/drive/MyDrive/IDP/Dataset/evaluation.dta', iterator=True)

# pegando as descrições das variáveis

descricoes = stata_reader.variable_labels()

descricoes

{'age_hh': 'Age of the head of the household (in years)',
 'age_sp': 'Age of the spouse (in years)',
 'bathroom': 'Home with private bathroom at baseline (0=no, 1=yes)',
 'dirtfloor': 'Home has a dirt floor at baseline (0=no, 1=yes)',
 'educ_hh': 'Education of the head of household (completed years of schooling)',
 'educ_sp': 'Education of the spouse (completed years of schooling)',
 'eligible': 'Household eligible to enroll in HISP (0=no, 1=yes)',
 'enrolled': 'HH enrolled in HISP (0=no, 1=yes)',
 'enrolled_rp': 'Household enrolled in HISP under the random promotion scenario (0=no, 1=yes)',
 'female_hh': 'Head of the household is a woman (0=no, 1=yes)',
 'health_expenditures': 'Out of pocket health expenditures (per person per year)',
 'hhsize': 'Number of household members (baseline)',
 'hospital': 'HH member visited hospital in the past year (0=no, 1=yes)',
 'hospital_distance': 'Distance to closest hospital',
 'household_identifier': 'Unique household identifier',
 'indigenous': 'H

In [48]:
# descrevendo o DataFrame

df.describe()

,locality_identifier,household_identifier,treatment_locality,promotion_locality,eligible,enrolled,enrolled_rp,poverty_index,round,health_expenditures,age_hh,age_sp,educ_hh,educ_sp,female_hh,indigenous,hhsize,dirtfloor,bathroom,land,hospital_distance,hospital
count,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,19827.000000,11257.000000
mean,73.933472,8038.965820,0.500277,0.512685,0.567761,0.299037,0.293287,56.789505,0.500025,17.035103,46.683117,40.581734,2.833550,2.618601,0.099057,0.352903,5.178645,0.603621,0.615978,2.079740,105.322517,0.052323
std,55.076599,4569.468262,0.500017,0.499851,0.495377,0.457822,0.455238,10.686106,0.500013,9.291589,15.294811,12.822810,2.754772,2.543370,0.298730,0.477869,2.195178,0.489157,0.486375,3.133202,42.063479,0.222696
min,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.479134,0.000000,0.000000,14.000000,14.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,9.465392,0.000000
25%,30.000000,3989.500000,0.000000,0.000000,0.000000,0.000000,0.000000,49.652241,0.000000,11.621977,34.000000,31.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,72.804218,0.000000
50%,59.000000,8040.000000,1.000000,1.000000,1.000000,0.000000,0.000000,56.414219,1.000000,16.051678,45.000000,41.000000,2.000000,2.000000,0.000000,0.000000,5.000000,1.000000,1.000000,1.000000,113.556497,0.000000
75%,112.000000,12033.000000,1.000000,1.000000,1.000000,1.000000,1.000000,62.905720,1.000000,21.236842,58.000000,48.000000,4.000000,4.000000,0.000000,1.000000,7.000000,1.000000,1.000000,3.000000,141.097735,0.000000
max,200.000000,15778.000000,1.000000,1.000000,1.000000,1.000000,1.000000,100.000000,1.000000,116.830643,88.000000,88.000000,16.000000,17.000000,1.000000,1.000000,13.000000,1.000000,1.000000,23.000000,170.457647,1.000000


### 1.2 - Divisão de antes e depois do programa

In [64]:
# dividindo de antes do programa

df_inscritos_antes = df.query('enrolled == 1 & round == 0 & treatment_locality == 1')

df_inscritos_antes

,locality_identifier,household_identifier,treatment_locality,promotion_locality,eligible,enrolled,enrolled_rp,poverty_index,round,health_expenditures,age_hh,age_sp,educ_hh,educ_sp,female_hh,indigenous,hhsize,dirtfloor,bathroom,land,hospital_distance,hospital
0,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,0.0,15.185455,24.0,23.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
2,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,0.0,13.076257,30.0,26.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,0.0
5,26.0,13.0,1.0,1.0,1.0,1.0,0.0,54.095825,0.0,15.286353,58.0,56.0,0.0,0.0,0.0,0.0,6.0,1,0,4,124.819966,0.0
7,26.0,16.0,1.0,1.0,1.0,1.0,1.0,56.903400,0.0,11.311761,35.0,24.0,3.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
8,26.0,21.0,1.0,1.0,1.0,1.0,1.0,46.908810,0.0,11.223912,37.0,35.0,0.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11247,40.0,15773.0,1.0,1.0,1.0,1.0,1.0,28.979908,0.0,7.218568,57.0,51.0,3.0,0.0,0.0,1.0,9.0,1,1,1,114.763392,0.0
11249,40.0,15774.0,1.0,1.0,1.0,1.0,1.0,28.593508,0.0,7.241524,53.0,53.0,5.0,0.0,0.0,1.0,9.0,1,1,2,114.763392,0.0
11252,40.0,15775.0,1.0,1.0,1.0,1.0,1.0,37.171562,0.0,10.391406,29.0,27.0,1.0,6.0,0.0,1.0,6.0,1,1,0,114.763392,0.0
11253,40.0,15776.0,1.0,1.0,1.0,1.0,1.0,29.057186,0.0,8.679602,78.0,41.0,0.0,2.0,0.0,1.0,9.0,1,0,0,114.763392,0.0


In [62]:
# dados de depois do programa

df_inscritos_depois = df.query('enrolled == 1 & round == 1')

df_inscritos_depois

,locality_identifier,household_identifier,treatment_locality,promotion_locality,eligible,enrolled,enrolled_rp,poverty_index,round,health_expenditures,age_hh,age_sp,educ_hh,educ_sp,female_hh,indigenous,hhsize,dirtfloor,bathroom,land,hospital_distance,hospital
1,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,1.0,19.580902,25.0,24.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
3,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,1.0,2.398854,31.0,27.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,1.0
4,26.0,13.0,1.0,1.0,1.0,1.0,0.0,54.095825,1.0,0.000000,59.0,57.0,0.0,0.0,0.0,0.0,6.0,1,0,4,124.819966,1.0
6,26.0,16.0,1.0,1.0,1.0,1.0,1.0,56.903400,1.0,20.026909,36.0,25.0,3.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
9,26.0,21.0,1.0,1.0,1.0,1.0,1.0,46.908810,1.0,16.664686,39.0,36.0,0.0,0.0,0.0,0.0,7.0,1,0,2,124.819966,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11248,40.0,15773.0,1.0,1.0,1.0,1.0,1.0,28.979908,1.0,0.000000,58.0,52.0,3.0,0.0,0.0,1.0,9.0,1,1,1,114.763392,0.0
11250,40.0,15774.0,1.0,1.0,1.0,1.0,1.0,28.593508,1.0,7.193629,54.0,54.0,5.0,0.0,0.0,1.0,9.0,1,1,2,114.763392,0.0
11251,40.0,15775.0,1.0,1.0,1.0,1.0,1.0,37.171562,1.0,0.727366,30.0,28.0,1.0,6.0,0.0,1.0,6.0,1,1,0,114.763392,0.0
11254,40.0,15776.0,1.0,1.0,1.0,1.0,1.0,29.057186,1.0,0.000000,79.0,41.0,0.0,2.0,0.0,1.0,9.0,1,0,0,114.763392,0.0


In [63]:
# dados de todos os inscritos no programa

df_inscritos = df.query('enrolled == 1')

df_inscritos

,locality_identifier,household_identifier,treatment_locality,promotion_locality,eligible,enrolled,enrolled_rp,poverty_index,round,health_expenditures,age_hh,age_sp,educ_hh,educ_sp,female_hh,indigenous,hhsize,dirtfloor,bathroom,land,hospital_distance,hospital
0,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,0.0,15.185455,24.0,23.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
1,26.0,5.0,1.0,1.0,1.0,1.0,1.0,55.950542,1.0,19.580902,25.0,24.0,0.0,6.0,0.0,0.0,4.0,1,0,1,124.819966,0.0
2,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,0.0,13.076257,30.0,26.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,0.0
3,26.0,11.0,1.0,1.0,1.0,1.0,0.0,46.058731,1.0,2.398854,31.0,27.0,4.0,0.0,0.0,0.0,6.0,1,0,2,124.819966,1.0
4,26.0,13.0,1.0,1.0,1.0,1.0,0.0,54.095825,1.0,0.000000,59.0,57.0,0.0,0.0,0.0,0.0,6.0,1,0,4,124.819966,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11252,40.0,15775.0,1.0,1.0,1.0,1.0,1.0,37.171562,0.0,10.391406,29.0,27.0,1.0,6.0,0.0,1.0,6.0,1,1,0,114.763392,0.0
11253,40.0,15776.0,1.0,1.0,1.0,1.0,1.0,29.057186,0.0,8.679602,78.0,41.0,0.0,2.0,0.0,1.0,9.0,1,0,0,114.763392,0.0
11254,40.0,15776.0,1.0,1.0,1.0,1.0,1.0,29.057186,1.0,0.000000,79.0,41.0,0.0,2.0,0.0,1.0,9.0,1,0,0,114.763392,0.0
11255,40.0,15777.0,1.0,1.0,1.0,1.0,1.0,38.485317,1.0,0.000000,32.0,26.0,4.0,2.0,0.0,1.0,6.0,1,0,0,114.763392,0.0


### 1.3 - Comparação de gastos médios

In [65]:
# realizando testes:

stats.ttest_ind(df_inscritos_antes['health_expenditures'], df_inscritos_depois['health_expenditures'], nan_policy='omit')

# o p-value é muito baixo, então rejeita-se a hipótese de que a média de gastos depois do programa é igual à de antes

Ttest_indResult(statistic=39.764560411867826, pvalue=9.90593763905028e-307)

## Questão 2

### 2.1 - Comparação das médias de antes e depois dos gastos com saúde

In [66]:
# média total

df['health_expenditures'].mean()

17.03510284423828

In [67]:
# média dos inscritos

df_inscritos['health_expenditures'].mean()

11.164377212524414

In [68]:
# média de antes

df_inscritos_antes['health_expenditures'].mean()

14.489691734313965

In [72]:
# desvio padrão de antes

statistics.stdev(df_inscritos_antes['health_expenditures'])

4.356316817636983

In [74]:
# intervalo de confiança

stats.norm.interval(alpha=0.95, loc=np.mean(df_inscritos_antes['health_expenditures']), scale=stats.sem(df_inscritos_antes['health_expenditures']))

(14.332861994093571, 14.646521474534358)

In [69]:
# média de depois

df_inscritos_depois['health_expenditures'].mean()

7.840174198150635

### 2.2 - Regressões

In [59]:
# regressão simples

formula_simples = "health_expenditures ~ 1 + round"

modelo_simples_antesdepois = ols(formula_simples, df_inscritos).fit(cov_type="HC2", use_t=True)

print(modelo_simples_antesdepois.summary())

# se pode deduzir que, com o passar do tempo, os gastos (em dólares por pessoa por ano) cairam $6,65

                             OLS Regression Results                            
Dep. Variable:     health_expenditures   R-squared:                       0.211
Model:                             OLS   Adj. R-squared:                  0.210
Method:                  Least Squares   F-statistic:                     1582.
Date:                 Fri, 21 Jan 2022   Prob (F-statistic):          8.84e-307
Time:                         02:12:44   Log-Likelihood:                -19453.
No. Observations:                 5929   AIC:                         3.891e+04
Df Residuals:                     5927   BIC:                         3.892e+04
Df Model:                            1                                         
Covariance Type:                   HC2                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     14.4897      0.080    181.08

In [60]:
# regressão com modelo mais completo

formula_completa = "health_expenditures ~ 1 + round + age_hh +  educ_hh + educ_sp + female_hh + indigenous + hhsize + dirtfloor + bathroom + land + hospital_distance"

modelo_completo_antesdepois = ols(formula_completa, df_inscritos).fit(cov_type="HC2", use_t=True)

print(modelo_completo_antesdepois.summary())

# o resultado é muito similar e com a mesma conclusão do modelo mais simples

                             OLS Regression Results                            
Dep. Variable:     health_expenditures   R-squared:                       0.483
Model:                             OLS   Adj. R-squared:                  0.482
Method:                  Least Squares   F-statistic:                     783.6
Date:                 Fri, 21 Jan 2022   Prob (F-statistic):               0.00
Time:                         02:15:05   Log-Likelihood:                -18201.
No. Observations:                 5929   AIC:                         3.643e+04
Df Residuals:                     5917   BIC:                         3.651e+04
Df Model:                           11                                         
Covariance Type:                   HC2                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            24.7391

## Questão 3